# Analysis

In [33]:
import os, re
from main.FolderInfos import FolderInfos

FolderInfos.init(test_without_data=True)
list_dataout_folders = os.listdir(FolderInfos.data_folder)
list_folders = []
for f in list_dataout_folders:
    full_path = FolderInfos.data_folder+f
    files = os.listdir(full_path)
    files_to_contain = [[re.compile(r".+_parameters.json$"),False]]
    for file in files:
        for i,check in enumerate(files_to_contain):
            if check[0].match(file):
                files_to_contain[i][1] = True
        if len(list(filter(lambda x:x[1] is False,files_to_contain))) == 0: # If all of the required files are in the folder
            list_folders.append(f)

print("\n".join(list_folders))
import ipywidgets as widgets
for f in list_folders:
    widgets.ToggleButton(
        value=False,
        description=f,
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='',
        icon='check' # (FontAwesome names without the `fa-` prefix)
    )
selected_folder = list_folders
import json
global_dict = {}
for f in selected_folder:
    full_path = FolderInfos.data_folder+f+FolderInfos.separator
    with open(f"{full_path}{f}parameters.json") as fp:
        global_dict[f] = json.load(fp)

2021-06-10_11h31min34s_


In [36]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

loss_dict = {}
for f in global_dict.keys():
    v = global_dict[f]["training"]["tr_loss"]
    loss_dict[f"Number of samples processed {f}"] = pd.Series(np.arange(0,len(v))*global_dict[f]["data"]["batch_size"])
    loss_dict[f"Training loss {f}"] = pd.Series(v)

import main.src.analysis.tools as tls
df = pd.DataFrame(loss_dict)
list_plots = []
window_mean = 10
max_length = 0
min_val,max_val = 1000000,-100000
for f in global_dict.keys():
    v = tls.moving_mean(global_dict[f]["training"]["tr_loss"],window=int(window_mean/global_dict[f]["data"]["batch_size"]))
    x=np.arange(0,len(v))*global_dict[f]["data"]["batch_size"]
    list_plots.append(go.Scatter(x=x,y=v,name=f,
                    mode='lines'))
    max_length = max(np.max(x),max_length)
    min_val = min(min_val,np.min(v))
    max_val = max(max_val,np.max(v))
fig = go.Figure(
    data=list_plots,
    layout=go.Layout(
        title=go.layout.Title(text="Loss evolution"),
        xaxis_title="Number of samples",
        yaxis_title=f"Loss (moving mean with a {window_mean} window"
    )
)
first_key = list(global_dict.keys())[0]
for x in np.arange(0,max_length,global_dict[first_key]["training"]["num_batches_tr"]*global_dict[first_key]["data"]["batch_size"]):
    fig.update_layout(shapes=[
        dict(
          type= 'line',
          yref= 'y', y0= min_val, y1= max_val,
          xref= 'x', x0= x, x1= x,
            line=dict(
                color="orange",
                width=4,
                dash="dashdot",
            )
        )
    ])
fig.update_yaxes(type="log")


fig.show()